<a href="https://colab.research.google.com/github/coldbilberry/repo-gui/blob/main/ML_in_Business_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

In [ ]:
df['gender'].value_counts('Male')

In [ ]:
df['InternetService'].nunique()

In [ ]:
df['TotalCharges'].describe()

In [ ]:
df.describe()

In [ ]:
df['PhoneService'].replace('Yes', 1, inplace=True)
df['PhoneService'].replace('No', 0, inplace=True)
df['PhoneService']

In [ ]:
df['TotalCharges'] = df['TotalCharges'].replace([' '], np.nan)
df['TotalCharges'] = df['TotalCharges'].astype(np.float32)
df['TotalCharges'].dtype

In [ ]:
df['TotalCharges'] = df['TotalCharges'].fillna(0)
df['TotalCharges'].describe()

In [ ]:
df['Churn'].replace('Yes', 1, inplace=True)
df['Churn'].replace('No', 0, inplace=True)
df['Churn']

In [ ]:
columns = ['StreamingMovies', 'StreamingTV', 'TechSupport']

for column in columns:
    df[column].replace('Yes', 1, inplace=True)
    df[column].replace('No', 0, inplace=True)
    df[column].replace('No internet service', 0, inplace=True)

df[['StreamingMovies', 'StreamingTV', 'TechSupport']]

In [ ]:
df['PhoneService'].value_counts()

In [ ]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges',
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']
#Ваш код здесь
columns_del = [x for x in df.columns if x not in columns]
columns_del

df.drop(columns_del, axis=1, inplace=True)
df.head(3)

In [ ]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'
#Ваш код здесь
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

In [ ]:
gender.fit_transform(X_train)
gender

In [ ]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

In [ ]:
TotalCharges =  Pipeline([
                ('selector', NumberSelector(key='TotalCharges')),
                ('standard', StandardScaler())
            ])

TotalCharges

In [ ]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV',
                                                 'TechSupport']))
            ])

In [ ]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

In [ ]:
pred_prob = pipeline.predict_proba(X_test)
pred_prob

In [ ]:
from sklearn.metrics import roc_auc_score, log_loss

#Ваш код здесь
pipeline.score(X_test, y_test)

In [ ]:
pred_prob[:,0]

In [ ]:
roc_auc_score(y_test, pred_prob[:,1])

In [ ]:
log_loss(y_test, pred_prob[:,1])

In [ ]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)